# Evaluate Hex Chess Engines

## Setup

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import os
import pickle
from tqdm import tqdm
from hexchess import HexChessBoard, Player, RandomPlayer, GreedyPlayer
from engines import QNetworkPlayer
from utils import make_safe_filename
import tensorflow as tf

/var/tmp/ipykernel_353624/1683085671.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was too old on your system - pyarrow 10.0.1 is the current minimum supported version as of this release.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


pygame 2.5.2 (SDL 2.28.2, Python 3.10.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


2024-04-06 18:11:38.466822: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-06 18:11:39.391640: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2024-04-06 18:11:39.391751: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such fi

In [3]:
gpus = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(gpus[0], True)
print(gpus[0])

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


2024-04-06 18:11:40.407473: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-06 18:11:40.422121: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-06 18:11:40.424924: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [4]:
# Constants
results_dir = "./assets/results"
os.makedirs(results_dir, exist_ok=True)

## Game Function

In [5]:
def play_game(white_player, black_player, board, max_moves = 50):
    # Reset board
    board.reset()

    # Play game
    n_moves = 0
    turn_is_white = True
    finished = False
    for _ in range(max_moves * 2):
        # Select active player
        active_player = white_player if turn_is_white else black_player

        # Get move
        position_from, position_to = active_player.get_move()
        success, finished = board.move(position_from, position_to, turn_is_white)

        # Update turn
        turn_is_white = not turn_is_white
        n_moves += 1 if turn_is_white else 0
        if finished:
            break

    # Determine winner (only valid if game is finished)
    white_won = not turn_is_white
    return finished, white_won, n_moves

In [6]:
def play_games(
    white_player_class,
    black_player_class,
    max_moves=50,
    n_games=10,
    progress_bar=True,
):
    # Parse player
    if isinstance(white_player_class, dict):
        white_player_kwargs = white_player_class['kwargs']
        white_player_class = white_player_class['class']
    else:
        white_player_kwargs = {}
    if isinstance(black_player_class, dict):
        black_player_kwargs = black_player_class['kwargs']
        black_player_class = black_player_class['class']
    else:
        black_player_kwargs = {}
    
    # Initialize board
    board = HexChessBoard()
    
    # Initialize players
    white_player = white_player_class(board, True, **white_player_kwargs)
    black_player = black_player_class(board, False, **black_player_kwargs)
    
    
    # Play games
    fn = tqdm if progress_bar else lambda x: x
    results = [None] * n_games
    for game_index in fn(range(n_games)):
        results[game_index] = play_game(
            white_player=white_player,
            black_player=black_player,
            board=board,
            max_moves=max_moves,
        )

    return results

## Evaluate Games

In [7]:
# Configuration
n_games = 1000
max_moves = 100
white_players = [
    {"class": RandomPlayer, "label": "Random (baseline)", "kwargs": {}}, 
    {"class": GreedyPlayer, "label": "Greedy (baseline)", "kwargs": {}},
    {"class": QNetworkPlayer, "label": "DQN small (random)", "kwargs": {"model_name": "random_pr", "is_large": False}},
    {"class": QNetworkPlayer, "label": "DQN small (random + greedy)", "kwargs": {"model_name": "randomgreedy_pr", "is_large": False}},
    {"class": QNetworkPlayer, "label": "DQN large (random)", "kwargs": {"model_name": "random_pr", "is_large": True}},
    {"class": QNetworkPlayer, "label": "DQN large (random + greedy)", "kwargs": {"model_name": "randomgreedy_pr", "is_large": True}},
    {"class": QNetworkPlayer, "label": "DQN large (random + greedy-2)", "kwargs": {"model_name": "randomgreedygreedy_pr", "is_large": True}},
]
n_players = len(white_players)
black_players = white_players[:n_players//4]

In [ ]:
# Gather results
rows = [player["label"] if isinstance(player, dict) else player.name for player in white_players]
cols = [player["label"] if isinstance(player, dict) else player.name for player in black_players]
games_won = np.zeros((len(white_players), len(black_players)), dtype=int)
games_draw = np.zeros((len(white_players), len(black_players)), dtype=int)
games_lost = np.zeros((len(white_players), len(black_players)), dtype=int)
games_moves = np.zeros((len(white_players), len(black_players)), dtype=float)

# All white players
for i, white_player in enumerate(white_players):
    # Against all black players
    for j, black_player in enumerate(black_players):
        # Battle start
        print(f"{white_player['label']} vs {black_player['label']}")
        
        # Make sure we don't already have the results
        results_filename = f"{make_safe_filename(white_player['label'])}_vs_{make_safe_filename(black_player['label'])}_{n_games}.pkl"
        results_filepath = os.path.join(results_dir, results_filename)
        if os.path.exists(results_filepath):
            # Results path exists - load from file
            with open(results_filepath, "rb") as results_file:
                results = pickle.load(results_file)
        else:
            # Results path does not exist - play games
            results = play_games(
                white_player,
                black_player,
                max_moves=max_moves,
                n_games=n_games,
                progress_bar=True,
            )
            # Save to file
            with open(results_filepath, "wb") as results_file:
                pickle.dump(results, results_file)
        
        # Save statistics
        for game_index, result in enumerate(results):
            # Parse result
            game_finished, white_won, n_moves = result
            
            # Win / Loss / Draw
            if game_finished:
                if white_won:
                    games_won[i, j] += 1
                else:
                    games_lost[i, j] += 1
            else:
                games_draw[i, j] += 1

            # Number of moves
            games_moves[i, j] += n_moves / n_games


Random (baseline) vs Random (baseline)


100%|███████████████████████████████████████| 1000/1000 [07:38<00:00,  2.18it/s]


Greedy (baseline) vs Random (baseline)


100%|███████████████████████████████████████| 1000/1000 [01:30<00:00, 11.11it/s]


DQN small (random) vs Random (baseline)


2024-04-06 18:20:50.635403: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-06 18:20:50.636775: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-06 18:20:50.639048: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-06 18:20:50.640896: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at 

In [29]:
# Collect results in Dataframes
games_won_df = pd.DataFrame(data = games_won, index = rows, columns = cols)
games_lost_df = pd.DataFrame(data = games_lost, index = rows, columns = cols)
games_draw_df = pd.DataFrame(data = games_draw, index = rows, columns = cols)
games_moves_df = pd.DataFrame(data = games_moves, index = rows, columns = cols)

In [30]:
# Save results
models = "qnetworks"
games_df_names = ["won", "lost", "draw", "moves"]
games_dfs = [games_won_df, games_lost_df, games_draw_df, games_moves_df]
for name, game_df in zip(games_df_names, games_dfs):
    game_df.to_csv(os.path.join(results_dir, f"games_{name}_{models}.csv"), index=True)

In [35]:
# Load results
models = "qnetworks"
games_df_names = ["won", "lost", "draw", "moves"]
games_dfs = [pd.read_csv(os.path.join(results_dir, f"games_{name}_{models}.csv"), index_col=0) for name in games_df_names]
games_won_df, games_lost_df, games_draw_df, games_moves_df = games_dfs

In [36]:
print("___GAMES WON___")
games_won_df

___GAMES WON___


,Random (baseline),Greedy (baseline),DQN (random),DQN (random + greedy)
Random (baseline),4,0,2,1
Greedy (baseline),10,9,10,9
DQN (random),4,2,0,0
DQN (random + greedy),3,2,0,0


In [37]:
print("___GAMES LOST___")
games_lost_df

___GAMES LOST___


,Random (baseline),Greedy (baseline),DQN (random),DQN (random + greedy)
Random (baseline),4,10,8,4
Greedy (baseline),0,1,0,1
DQN (random),6,8,0,0
DQN (random + greedy),5,8,10,0


In [38]:
print("___GAMES DRAW___")
games_draw_df

___GAMES DRAW___


,Random (baseline),Greedy (baseline),DQN (random),DQN (random + greedy)
Random (baseline),2,0,0,5
Greedy (baseline),0,0,0,0
DQN (random),0,0,10,10
DQN (random + greedy),2,0,0,10


In [39]:
print("__NR OF MOVES__")
games_moves_df

__NR OF MOVES__


,Random (baseline),Greedy (baseline),DQN (random),DQN (random + greedy)
Random (baseline),46.5,10.9,38.0,70.2
Greedy (baseline),9.5,19.0,10.5,19.3
DQN (random),53.4,10.0,100.0,100.0
DQN (random + greedy),51.6,15.6,5.0,100.0
